In [ ]:
import numpy as np
import pandas as pd
import re
import glob

In [ ]:
# Function that takes in the JSON File and return a list of tuples where each tuple is (Run_Config, Omnetpp.ini, TargetsDictionary)

def JSONExtractor(JSON):
  runINI = []
  delINI = []
  ignore = []
  disk = "'peakdiskspaceusedbytes'"
  simRAM = "'peakramusedsimbytes'"
  resRAM = "'peakramusedresultsbytes'"
  time = "'totaljobclocktimesec'"

  for i in range(len(JSON)):
    
    # Checking if the Simulation_Completion and Results_Parsing_Completion are 100% and current state = 'COMPLETED'
    if ((JSON.simulations[i]['meta']['sim_completed_perc']==100) and (JSON.simulations[i]['meta']['results_completed_perc']==100) and (JSON.simulations[i]['meta']['current_state']=='COMPLETED')):

      # Checking if simulation contains meta data with 'sim_runtime_stats' as key (if not present 'totaljobclocktimesec' param can't be found, so discarding them)
      if 'sim_runtime_stats' in JSON.simulations[i]['meta']:

        # Fetching the all 4 targets variables from meta data of the simulation 
        dicts = {'peakdiskspaceusedbytes':re.compile(disk + r'\,\s([0-9\.]+)').findall(str(JSON.simulations[i]['meta']['sim_runtime_stats'])),
             'peakramusedsimbytes':re.compile(simRAM + r'\,\s([0-9\.]+)').findall(str(JSON.simulations[i]['meta']['sim_runtime_stats'])),
             'peakramusedresultsbytes':re.compile(resRAM + r'\,\s([0-9\.]+)').findall(str(JSON.simulations[i]['meta']['sim_runtime_stats'])),                                                                      
             'totaljobclocktimesec':re.compile(time + r'\,\s([0-9\.]+)').findall(str(JSON.simulations[i]['meta']['sim_runtime_stats'])) }
        
        # Appending Run_Config, Omnetpp.ini and Targets dictionary as a tuple in runINI list
        runINI.append((JSON.simulations[i]['runconfig'],JSON.simulations[i]['omnetppini'],dicts))
      else:
        ignore.append((JSON.simulations[i]['runconfig'],JSON.simulations[i]['omnetppini']))
    else:
      # print(i)
      # Appending info about incomplete simulations
      delINI.append((JSON.simulations[i]['runconfig'],JSON.simulations[i]['omnetppini'],dicts))

  return runINI

In [ ]:
# Function that takes in Run_Config, .ini, TargetsDictionary and returns a dictionary of all .ini parameters 
# of a simulation along with their corresponding targets

def FeatureExtractor(runconfig,omnetppini,targets):
  if not runconfig == 'General':
    # print(runconfig)
    # replacing multiple \r and \n with null and \n resp. and splitting on \n basis
    splitINI = re.sub(r'[\n]+','\n ',re.sub(r'[\r]+','',omnetppini)).strip().split('\n')

    # INI filtering (removes any redundant commands/comments starting with '#' in the ini) 
    refreshINI = ''.join([line.strip() for line in splitINI if not line.strip().startswith('#')])

    # RunConfig Filtering (Filter INI for the specific RunConfig) --- case where mutiple Run_Configs 
    # are present in same ini file
    configFiltered = re.compile(runconfig + r'[\'\]\n\#\s\w\*\.=\(\)\"\,\-\/\+]+').findall(refreshINI)

    # Extracting every ini param and converting to dict
    paramDict = dict(re.findall(r'\*\*\.([a-zA-Z\.]+) = ([A-Za-z0-9\"\.\/\-]+)',str(configFiltered)))
    if 'numNodes' not in paramDict.keys():
      x = re.findall(r'\*\*\.(numNodes) = ([0-9]+)',refreshINI)[0]
      paramDict[x[0]] = x[1]

    # Combining .ini parameters dictionary with TargetsDictionary
    paramDict.update(targets)

    return paramDict

In [ ]:
JSONDrives = glob.glob('/content/drive/MyDrive/OOTB/simulation-meta_26_02_2023.json')
print(len(JSONDrives))
JSONDrives

In [ ]:
finalDic = []
inis = []
runconfig = []
for path in JSONDrives:
  try:
    json = pd.read_json(path)
    print(JSONDrives[-1])
    JsonFiltered = JSONExtractor(json)
    # print("Initial Length of JSON",len(json),"Final Length of Filtered JSON",len(JsonFiltered))

    # ParamsList = []
    for rc,ini,targets in JsonFiltered:
      # Feature Extractor function returns .ini params and targets as dictionary
      dic = FeatureExtractor(rc,ini,targets)
      if bool(dic):
        # ParamsList.append(dic)
        inis.append(ini)
        runconfig.append(rc)
        finalDic.append(dic)
  except:
    # print(path)
    print("ss")

In [ ]:
Numericals = ['app_dataGenerationInterval','constraintAreaMaxX','constraintAreaMaxY','mobility_noOfLocations','mobility_Hosts','mobility_speed','numNodes','app_dataSizeInBytes','forwarding_maximumCacheSize','mobility_nodeId']
Categoricals = ['applicationLayer','forwardingLayer','linkLayer','mobilityType']
Targets = ['peakdiskspaceusedbytes','peakramusedsimbytes','peakramusedresultsbytes','totaljobclocktimesec']

In [ ]:
first = pd.DataFrame(finalDic)
first

In [ ]:
for_null = first[first['forwardingLayer'].isna()]
for_null

In [ ]:
second = first[~first['forwardingLayer'].isna()]
second

In [ ]:
second.columns = second.columns.str.replace('.','_',regex=False)

remove_cols = ['app_popularityAssignmentPercentage','app_usedRNG','forwarding_antiEntropyInterval','forwarding_maximumHopCount','forwarding_usedRNG','link_wirelessRange','link_neighbourScanInterval','link_bandwidthBitRate','link_wirelessHeaderSize','link_usedRNG','constraintAreaMinX','constraintAreaMinY','constraintAreaMinZ','constraintAreaMaxZ','updateInterval','mobility_initFromDisplayString','mobility_popularityDecisionThreshold','mobility_returnHomePercentage','mobility_neighbourLocationLimit','mobility_radius','mobility_alpha','mobility_waitTime','mobility_usedRNG','app_dataGenerationIntervalMode','app_trafficInfoPath','forwarding_spraywaitFlavour','forwarding_noDuplicate','mobility_traceFile','link_contactTracesPath','forwarding_broadcastRRS','forwarding_sendOnNeighReportingFrequency','forwarding_sendFrequencyWhenNotOnNeighFrequency','forwarding_pEncounterMax','forwarding_pEncounterFirst','forwarding_pFirstThreshold','forwarding_alpha','forwarding_beta','forwarding_gamma','forwarding_delta','forwarding_standardTimeInterval','forwarding_agingInterval','forwarding_neighbourhoodChangeSignificanceThreshold','forwarding_coolOffDuration','forwarding_learningConst','forwarding_backoffTimerIncrementFactor','app_specificDestination','app_specificDestinationNodeName','app_ttl','forwarding_useTTL']

leftover_cols = [col for col in second.columns if col not in remove_cols]

# Different coulmn categories
Numericals = ['app_dataGenerationInterval','constraintAreaMaxX','constraintAreaMaxY','mobility_noOfLocations','mobility_Hosts','mobility_speed','numNodes','app_dataSizeInBytes','forwarding_maximumCacheSize','mobility_nodeId']
Categoricals = ['applicationLayer','forwardingLayer','linkLayer','mobilityType']
Targets = ['peakdiskspaceusedbytes','peakramusedsimbytes','peakramusedresultsbytes','totaljobclocktimesec']

# Seprating dataframe with required columns
third = second[leftover_cols]

In [ ]:
third.mobilityType = third.mobilityType.fillna("ContactTraces")
third.mobility_nodeId = third.mobility_nodeId.fillna('0').str.replace('-1','1')
third.constraintAreaMaxX = third.constraintAreaMaxX.fillna('0')
third.constraintAreaMaxY = third.constraintAreaMaxY.fillna('0')
third.mobility_noOfLocations = third.mobility_noOfLocations.fillna('0')
third.mobility_Hosts = third.mobility_Hosts.fillna('0')
third.mobility_speed = third.mobility_speed.fillna('0')
third.forwarding_maximumCacheSize = third.forwarding_maximumCacheSize.fillna('0')

In [ ]:
one_hots = pd.get_dummies(third[Categoricals],drop_first=True)
fourth = pd.concat([third,one_hots],axis=1)
fourth = fourth.drop(Categoricals,axis=1)

In [ ]:
# Changing Numerical features from strings to integers
fourth['app_dataGenerationInterval'] =  fourth.app_dataGenerationInterval.apply(lambda x : x.replace('s','') if 's' in x else x)
fourth['constraintAreaMaxX'] =  fourth.constraintAreaMaxX.apply(lambda x : x.replace('m','') if 'm' in x else x)
fourth['constraintAreaMaxY'] =  fourth.constraintAreaMaxY.apply(lambda x : x.replace('m','') if 'm' in x else x)
fourth['mobility_speed'] =  fourth.mobility_speed.apply(lambda x : x.replace('mps','') if 'mps' in x else x)
def for_maxCache(cache):
  if 'bytes' in str(cache):
    return cache.replace('bytes','')
  elif 'byte' in str(cache):
    return cache.replace('byte','')
  else:
    return cache
fourth['forwarding_maximumCacheSize'] =  fourth.forwarding_maximumCacheSize.apply(lambda x : for_maxCache(x))

In [ ]:
#Separating features from the dataset and changing their datatype
features = fourth.drop(Targets,axis=1)
fourth[features.columns] = features.apply(pd.to_numeric)

# Changing Target variables from string to integers
def target_list(target):
  if type(target) is list:
    return pd.to_numeric(target[0],errors='coerce')
for col in Targets:
  fourth[col] = fourth[col].apply(lambda x: target_list(x))

In [ ]:
fourth.columns = fourth.columns.str.replace('"','')
fourth = fourth.reset_index(drop=True)
fourth

In [ ]:
#Save the dataset into the drive
# fourth.to_csv('/content/drive/MyDrive/OOTB/DataWithDummy_26_02_2023.csv',index=False)

Pre-Processing DATA

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew, johnsonsu, lognorm
from scipy.special import boxcox1p,inv_boxcox1p

In [ ]:
fifth.columns

In [ ]:
# MobilityNodeID=1 for BONN_MOTION_MOBILITY simualtions
sns.histplot(fifth['mobility_nodeId'],stat='count')

In [ ]:
## Skewness of all features
# Check the skew of all numerical features
skewed_feats = fifth[Numericals].apply(lambda x: skew(x)).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness

In [ ]:
## Applying BoxCox transforms
skewness = skewness[abs(skewness['Skew'].values) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
  print(feat)
  #all_data[feat] += 1
  fifth[feat] = boxcox1p(fifth[feat], lam)

In [ ]:
# Check the NEW skew of all numerical features
skewed_feats = fifth[Numericals].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness

In [ ]:
sixth = fifth.copy(deep=True)
sixth

Modelling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout
from keras.optimizers import Adam,SGD,RMSprop,Adadelta,Adagrad,Adamax,Nadam,Ftrl
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier,KerasRegressor
from keras.utils import plot_model
from sklearn.metrics import make_scorer,accuracy_score
from sklearn.model_selection import StratifiedKFold,KFold
from keras.layers import LeakyReLU
from keras.losses import mean_squared_error,mean_squared_logarithmic_error,mean_absolute_error,huber_loss
from keras.callbacks import LearningRateScheduler,EarlyStopping
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb
from sklearn.base import BaseEstimator, TransformerMixin,RegressorMixin,clone
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import time
import pickle
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [ ]:
X = sixth.drop(Targets,axis=1)
y = boxcox1p(sixth[Targets],0.20)

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.1,shuffle=True)

train_scalar1 = StandardScaler()

train_scalar2 = StandardScaler()

x_train_scaled = pd.DataFrame(data=train_scalar1.fit_transform(x_train),columns=x_train.columns,index=x_train.index)
y_train_scaled = pd.DataFrame(train_scalar2.fit_transform(y_train),columns=y_train.columns,index=y_train.index)

x_test_scaled = pd.DataFrame(data=train_scalar1.transform(x_test),columns=x_test.columns,index=x_test.index)
y_test_scaled = pd.DataFrame(train_scalar2.transform(y_test),columns=y_test.columns,index=y_test.index)

x_train_scaled,x_val_scaled,y_train_scaled,y_val_scaled = train_test_split(x_train_scaled,y_train_scaled,test_size=0.1)

In [ ]:
print("Samples in Train set : ", len(x_train_scaled))
print("Samples in Validation set : ", len(x_val_scaled))
print("Samples in Test set : ", len(x_test_scaled))

In [ ]:
print("Shape of Train set : ", x_train_scaled.shape)
print("Shape of Validation set : ", x_val_scaled.shape)
print("Shape of Test set : ", x_test_scaled.shape)

In [ ]:
# # Target distribution in Train, Validation and Test sets

# sns.distplot(y_train_scaled.iloc[:,1],hist=False,label='Train data')
# # sns.distplot(y_test.iloc[:,1],hist=False, label= 'Test data')
# sns.distplot(y_val_scaled.iloc[:,1],hist=False, label= 'Validation data')
# plt.xlabel('Peak DISK Usage (scaled)')
# plt.legend()

In [ ]:
# Stats of targets in test dataset
print(len(y_test))
print(y_test.iloc[:,0].mean())
print(y_test.iloc[:,0].std())

MODEL 1 (NEURAL NETWORK)

In [ ]:
# SINGLE NEURAL NET
single = Sequential()
single.add(Dense(units=40,input_dim=x_train_scaled.shape[1],activation='relu'))
single.add(Dropout(0.1))
single.add(Dense(20, activation='relu'))
single.add(Dropout(0.1))
single.add(Dense(12, activation='relu'))
single.add(Dropout(0.1))
single.add(Dense(8, activation='relu'))
single.add(Dropout(0.1))
single.add(Dense(4))
single.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.1),metrics=['mean_squared_error'])

def scheduler(epoch, lr):
  if epoch < 2:
    return lr
  else:
    return lr * np.exp(-0.1)
callbacks = [EarlyStopping(monitor='loss', patience=4),LearningRateScheduler(scheduler)]
single.fit(x_train_scaled,y_train_scaled,epochs=100,callbacks=callbacks)

In [ ]:
# SINGLE NEURAL NET
single_pred = pd.DataFrame(inv_boxcox1p(train_scalar2.inverse_transform(single.predict(x_test_scaled)),0.20),columns=y_test_scaled.columns,index=y_test_scaled.index)
y_test_org  = inv_boxcox1p(y_test,0.20)
single_pred.head()

In [ ]:
# PERFORMANCE METRICS of the SINGLE NEURAL NETWORK model
y1 = mean_absolute_error(y_test_org.iloc[:,0],single_pred.iloc[:,0])
print(y1*(1e-9))
y2 = mean_absolute_error(y_test_org.iloc[:,1],single_pred.iloc[:,1])
print(y2*(1e-9))
y3 = mean_absolute_error(y_test_org.iloc[:,2],single_pred.iloc[:,2])
print(y3*(1e-9))
y4 = mean_absolute_error(y_test_org.iloc[:,3],single_pred.iloc[:,3])
print(y4)

In [ ]:
class CustomEstimator(BaseEstimator,TransformerMixin):
  def __init__(self):
    self.model1 = KerasRegressor(build_fn=self.base1,epochs=100,batch_size=64,verbose=0)
    self.model2 = KerasRegressor(build_fn=self.base2,epochs=100,batch_size=64,verbose=0)
    self.model3 = KerasRegressor(build_fn=self.base3,epochs=100,batch_size=64,verbose=0)
    self.model4 = KerasRegressor(build_fn=self.base4,epochs=100,batch_size=64,verbose=0)

  # def fit(self,X,y):
  #   history1 = self.model1.fit(X,y[0])
  #   history2 = self.model2.fit(X,y[1])
  #   history3 = self.model3.fit(X,y[2])
  #   history4 = self.model4.fit(X,y[3])
  def fit(self,X,y,x_val_scaled,y_val_scaled):
    history1 = self.model1.fit(X,y[0],validation_data=(x_val_scaled,y_val_scaled))
    history2 = self.model2.fit(X,y[1],validation_data=(x_val_scaled,y_val_scaled))
    history3 = self.model3.fit(X,y[2],validation_data=(x_val_scaled,y_val_scaled))
    history4 = self.model4.fit(X,y[3],validation_data=(x_val_scaled,y_val_scaled))
    return self,[history1,history2,history3,history4]

  def predict(self,x_test):
    p1 = pd.DataFrame(self.model1.predict(x_test))
    p2 = pd.DataFrame(self.model2.predict(x_test))
    p3 = pd.DataFrame(self.model3.predict(x_test))
    p4 = pd.DataFrame(self.model4.predict(x_test))
    pred_df = pd.concat([p1,p2,p3,p4],axis=1)

    return pred_df
  
  def base1(self):
    model1 = Sequential()
    model1.add(Dense(units=44,input_dim=x_train_scaled.shape[1],activation='relu'))
    model1.add(Dropout(0.1))
    model1.add(Dense(22, activation='relu'))
    model1.add(Dropout(0.1))
    model1.add(Dense(12, activation='relu'))
    model1.add(Dropout(0.1))
    model1.add(Dense(8, activation='relu'))
    model1.add(Dropout(0.1))
    model1.add(Dense(4, activation='relu'))
    model1.add(Dropout(0.1))
    model1.add(Dense(1))
    model1.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.01),metrics=['mean_squared_error'])
    return model1

  def base2(self):
    model2 = Sequential()
    model2.add(Dense(units=44,input_dim=x_train_scaled.shape[1],activation='relu'))
    model2.add(Dropout(0.1))
    model2.add(Dense(22, activation='relu'))
    model2.add(Dropout(0.1))
    model2.add(Dense(12, activation='relu'))
    model2.add(Dropout(0.1))
    model2.add(Dense(8, activation='relu'))
    model2.add(Dropout(0.1))
    model2.add(Dense(4, activation='relu'))
    model2.add(Dropout(0.1))
    model2.add(Dense(1))
    model2.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.01),metrics=['mean_squared_error'])
    return model2

  def base3(self):
    model3 = Sequential()
    model3.add(Dense(units=44,input_dim=x_train_scaled.shape[1],activation='relu'))
    model3.add(Dropout(0.1))
    model3.add(Dense(22, activation='relu'))
    model3.add(Dropout(0.1))
    model3.add(Dense(12, activation='relu'))
    model3.add(Dropout(0.1))
    model3.add(Dense(8, activation='relu'))
    model3.add(Dropout(0.1))
    model3.add(Dense(4, activation='relu'))
    # model3.add(Dropout(0.1))
    model3.add(Dense(1))
    model3.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.01),metrics=['mean_squared_error'])
    return model3

  def base4(self):
    model4 = Sequential()
    model4.add(Dense(units=44,input_dim=x_train_scaled.shape[1],activation='relu'))
    model4.add(Dropout(0.1))
    model4.add(Dense(22, activation='relu'))
    model4.add(Dropout(0.1))
    model4.add(Dense(12, activation='relu'))
    model4.add(Dropout(0.1))
    model4.add(Dense(8, activation='relu'))
    model4.add(Dropout(0.1))
    model4.add(Dense(4, activation='relu'))
    model4.add(Dropout(0.1))
    model4.add(Dense(1))
    model4.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.01),metrics=['mean_squared_error'])
    return model4

In [ ]:
CE = CustomEstimator()
start_time = time.time()
# _ , histories = CE.fit(x_train_scaled,[y_train_scaled.iloc[:,0].values,y_train_scaled.iloc[:,1].values,y_train_scaled.iloc[:,2].values,y_train_scaled.iloc[:,3].values])
_ , histories = CE.fit(x_train_scaled,[y_train_scaled.iloc[:,0].values,y_train_scaled.iloc[:,1].values,y_train_scaled.iloc[:,2].values,y_train_scaled.iloc[:,3].values],x_val_scaled,y_val_scaled)
end_time = time.time()
print(end_time-start_time)

In [ ]:
train_loss = pd.DataFrame(inv_boxcox1p(train_scalar2.inverse_transform(pd.DataFrame(np.hstack((np.array(histories[0].history['loss']).reshape(-1,1),np.array(histories[1].history['loss']).reshape(-1,1),np.array(histories[2].history['loss']).reshape(-1,1),np.array(histories[3].history['loss']).reshape(-1,1))))),0.20)).multiply(1e-9)
val_loss = pd.DataFrame(inv_boxcox1p(train_scalar2.inverse_transform(pd.DataFrame(np.hstack((np.array(histories[0].history['val_loss']).reshape(-1,1),np.array(histories[1].history['val_loss']).reshape(-1,1),np.array(histories[2].history['val_loss']).reshape(-1,1),np.array(histories[3].history['val_loss']).reshape(-1,1))))),0.20)).multiply(1e-9)

train_loss_time = pd.DataFrame(inv_boxcox1p(train_scalar2.inverse_transform(pd.DataFrame(np.hstack((np.array(histories[0].history['loss']).reshape(-1,1),np.array(histories[1].history['loss']).reshape(-1,1),np.array(histories[2].history['loss']).reshape(-1,1),np.array(histories[3].history['loss']).reshape(-1,1))))),0.20))
val_loss_time = pd.DataFrame(inv_boxcox1p(train_scalar2.inverse_transform(pd.DataFrame(np.hstack((np.array(histories[0].history['val_loss']).reshape(-1,1),np.array(histories[1].history['val_loss']).reshape(-1,1),np.array(histories[2].history['val_loss']).reshape(-1,1),np.array(histories[3].history['val_loss']).reshape(-1,1))))),0.20))

In [ ]:
# Epoch vs Loss Plots for Target Variables

plt.rcParams["figure.figsize"] = (20,10)
fig,axs = plt.subplots(2,2)

axs[0,0].plot(train_loss.iloc[:,0])
axs[0,0].plot(val_loss.iloc[:,0])
axs[0,0].set_title('Peak Disk Usage')
axs[0,0].set_xlabel('epochs')
axs[0,0].set_ylabel('MSE Loss in GB')
axs[0,0].legend(['train', 'val'], loc='upper left')

axs[0,1].plot(train_loss.iloc[:,1])
axs[0,1].plot(val_loss.iloc[:,1])
axs[0,1].set_title('Peak Simulation RAM')
axs[0,1].set_xlabel('epochs')
axs[0,1].set_ylabel('MSE Loss in GB')
axs[0,1].legend(['train', 'val'], loc='upper left')

axs[1,0].plot(train_loss.iloc[:,2])
axs[1,0].plot(val_loss.iloc[:,2])
axs[1,0].set_title('Peak Results Parsing RAM')
axs[1,0].set_xlabel('epochs')
axs[1,0].set_ylabel('MSE Loss in GB')
axs[1,0].legend(['train', 'val'], loc='upper left')

axs[1,1].plot(train_loss_time.iloc[:,3])
axs[1,1].plot(val_loss_time.iloc[:,3])
axs[1,1].set_title('Total Time Taken')
axs[1,1].set_xlabel('epochs')
axs[1,1].set_ylabel('MSE Loss in Secs')
axs[1,1].legend(['train', 'val'], loc='upper left')

# plt.set_title('Training/Validation plots')
fig.show()

In [ ]:
x = CE.predict(x_test_scaled)

In [ ]:
CE_df = pd.DataFrame(inv_boxcox1p(train_scalar2.inverse_transform(x),0.20),columns=y_test_scaled.columns,index=y_test_scaled.index)
CE_df.head()

In [ ]:
y_test_org  = inv_boxcox1p(y_test,0.20)
y_test_org.head()

In [ ]:
# PERFORMANCE METRICS of the model
y1 = mean_absolute_error(y_test_org.iloc[:,0],CE_df.iloc[:,0])
print(y1*(1e-9))
y2 = mean_absolute_error(y_test_org.iloc[:,1],CE_df.iloc[:,1])
print(y2*(1e-9))
y3 = mean_absolute_error(y_test_org.iloc[:,2],CE_df.iloc[:,2])
print(y3*(1e-9))
y4 = mean_absolute_error(y_test_org.iloc[:,3],CE_df.iloc[:,3])
print(y4)

In [ ]:
# PERFORMANCE METRICS of the model
y1 = np.sqrt(mean_squared_error(y_test_org.iloc[:,0],CE_df.iloc[:,0]))
print(y1*(1e-9))
y2 = np.sqrt(mean_squared_error(y_test_org.iloc[:,1],CE_df.iloc[:,1]))
print(y2*(1e-9))
y3 = np.sqrt(mean_squared_error(y_test_org.iloc[:,2],CE_df.iloc[:,2]))
print(y3*(1e-9))
y4 = np.sqrt(mean_squared_error(y_test_org.iloc[:,3],CE_df.iloc[:,3]))
print(y4)

In [ ]:
#Save the  model
# pickle.dump(CE,open('NNCustEst.pkl','wb'))
# pickle.dump(train_scalar2,open('CE_TargetScalar.pkl','wb'))
# pickle.dump(train_scalar1,open('CE_FeatureScalar.pkl','wb'))

MODEL 2 (AVERAGING MODEL)

In [ ]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.01,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='squared_error', random_state =5)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.01, max_depth=3, 
                             min_child_weight=1.5, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)

model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.01, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        self.models2 = models
        self.models3 = models
        self.models4 = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        self.models2_ = [clone(x) for x in self.models2]
        self.models3_ = [clone(x) for x in self.models3]
        self.models4_ = [clone(x) for x in self.models4]
        self.clone_models = [self.models_,self.models2_,self.models3_,self.models4_]
        # Train cloned base models
        for model,model2,model3,model4 in zip(self.models_,self.models2_,self.models3_,self.models4_):
            model.fit(X, y[0])
            model2.fit(X, y[1])
            model3.fit(X, y[2])
            model4.fit(X, y[3])
        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        predictions2 = np.column_stack([
            model.predict(X) for model in self.models2_
        ])
        predictions3 = np.column_stack([
            model.predict(X) for model in self.models3_
        ])
        predictions4 = np.column_stack([
            model.predict(X) for model in self.models4_
        ])
        return [np.mean(predictions, axis=1),np.mean(predictions2, axis=1) ,np.mean(predictions3, axis=1) ,np.mean(predictions4, axis=1)]

In [ ]:
CE2 = AveragingModels((model_xgb, GBoost, model_lgb))
start_time = time.time()
_ = CE2.fit(x_train_scaled,[y_train_scaled.iloc[:,0],y_train_scaled.iloc[:,1],y_train_scaled.iloc[:,2],y_train_scaled.iloc[:,3]])
end_time = time.time()
print(end_time-start_time)

In [ ]:
CE2_df = pd.DataFrame(inv_boxcox1p(train_scalar2.inverse_transform(pd.DataFrame(CE2.predict(x_test_scaled)).transpose()),0.20),columns=y_test_scaled.columns,index=y_test_scaled.index)
CE2_df.head()

In [ ]:
y_test_org = inv_boxcox1p(y_test,0.20)
y_test_org.head()

In [ ]:
y1 = mean_absolute_error(y_test_org.iloc[:,0],CE2_df.iloc[:,0])
print(y1*(1e-9))
y2 = mean_absolute_error(y_test_org.iloc[:,1],CE2_df.iloc[:,1])
print(y2*(1e-9))
y3 = mean_absolute_error(y_test_org.iloc[:,2],CE2_df.iloc[:,2])
print(y3*(1e-9))
y4 = mean_absolute_error(y_test_org.iloc[:,3],CE2_df.iloc[:,3])
print(y4)

In [ ]:
# PERFORMANCE METRICS of the model
y1 = np.sqrt(mean_squared_error(y_test_org.iloc[:,0],CE2_df.iloc[:,0]))
print(y1*(1e-9))
y2 = np.sqrt(mean_squared_error(y_test_org.iloc[:,1],CE2_df.iloc[:,1]))
print(y2*(1e-9))
y3 = np.sqrt(mean_squared_error(y_test_org.iloc[:,2],CE2_df.iloc[:,2]))
print(y3*(1e-9))
y4 = np.sqrt(mean_squared_error(y_test_org.iloc[:,3],CE2_df.iloc[:,3]))
print(y4)

In [ ]:
# pickle.dump(CE2,open('AVGCustEst.pkl','wb'))
# pickle.dump(train_scalar2,open('CE_TargetScalar.pkl','wb'))
# pickle.dump(train_scalar1,open('CE_FeatureScalar.pkl','wb'))

Combined Model

In [ ]:
probs = np.linspace(0.1,1,10)[:-1]
probs

In [ ]:
DISK_maes = []
SIM_maes = []
RES_maes = []
TIME_maes = []

for prob in probs:
  temp_mix = CE_df.multiply(prob).add(CE2_df.multiply(1-prob))
  DISK_maes.append(mean_absolute_error(y_test_org.iloc[:,0],temp_mix.iloc[:,0]))
  SIM_maes.append(mean_absolute_error(y_test_org.iloc[:,1],temp_mix.iloc[:,1]))
  RES_maes.append(mean_absolute_error(y_test_org.iloc[:,2],temp_mix.iloc[:,2]))
  TIME_maes.append(mean_absolute_error(y_test_org.iloc[:,3],temp_mix.iloc[:,3]))

In [ ]:
Final_Mix = CE_df.multiply(0.3).add(CE2_df.multiply(0.7))
Final_Mix.head()

In [ ]:
print(abs((y_test_org.iloc[:,0] - CE_df.iloc[:,0]).mean()))
print(abs((y_test_org.iloc[:,0] - CE2_df.iloc[:,0]).mean()))
print(abs((y_test_org.iloc[:,0] - Final_Mix.iloc[:,0]).mean()))

In [ ]:
print('CE_df peakdisk',r2_score(y_test_org.iloc[:,0],CE_df.iloc[:,0]))
print('CE2_df peakdisk',r2_score(y_test_org.iloc[:,0],CE2_df.iloc[:,0]))
print('Final_Mix peakdisk',r2_score(y_test_org.iloc[:,0],Final_Mix.iloc[:,0]))
print(' ')
print('CE_df peakSim',r2_score(y_test_org.iloc[:,1],CE_df.iloc[:,1]))
print('CE2_df peakSim',r2_score(y_test_org.iloc[:,1],CE2_df.iloc[:,1]))
print('Final_Mix peakSim',r2_score(y_test_org.iloc[:,1],Final_Mix.iloc[:,1]))
print(' ')
print('CE_df peakRes',r2_score(y_test_org.iloc[:,2],CE_df.iloc[:,2]))
print('CE2_df peakRes',r2_score(y_test_org.iloc[:,2],CE2_df.iloc[:,2]))
print('Final_Mix peakRes',r2_score(y_test_org.iloc[:,2],Final_Mix.iloc[:,2]))
print(' ')
print('CE_df totalTime',r2_score(y_test_org.iloc[:,3],CE_df.iloc[:,3]))
print('CE2_df totalTime',r2_score(y_test_org.iloc[:,3],CE2_df.iloc[:,3]))
print('Final_Mix totalTime',r2_score(y_test_org.iloc[:,3],Final_Mix.iloc[:,3]))

In [ ]:
x = ['RandomForest','XGBoost','NeuralNetwork','AverageRegressor','Combined']
y1 = [20830591385.6,12745315450.88,mean_absolute_error(y_test_org.iloc[:,0],CE_df.iloc[:,0]),mean_absolute_error(y_test_org.iloc[:,0],CE2_df.iloc[:,0]),mean_absolute_error(y_test_org.iloc[:,0],Final_Mix.iloc[:,0])]
y1 = [y*(1e-9) for y in y1]
y2 = [3221225472,5905580032,mean_absolute_error(y_test_org.iloc[:,1],CE_df.iloc[:,1]),mean_absolute_error(y_test_org.iloc[:,1],CE2_df.iloc[:,1]),mean_absolute_error(y_test_org.iloc[:,1],Final_Mix.iloc[:,1])]
y2 = [y*(1e-9) for y in y2]
y3 = [4294967296,3435973836.8,mean_absolute_error(y_test_org.iloc[:,2],CE_df.iloc[:,2]),mean_absolute_error(y_test_org.iloc[:,2],CE2_df.iloc[:,2]),mean_absolute_error(y_test_org.iloc[:,2],Final_Mix.iloc[:,2])]
y3 = [y*(1e-9) for y in y3]
y4 = [4300,5000,mean_absolute_error(y_test_org.iloc[:,3],CE_df.iloc[:,3]),mean_absolute_error(y_test_org.iloc[:,3],CE2_df.iloc[:,3]),mean_absolute_error(y_test_org.iloc[:,3],Final_Mix.iloc[:,3])]

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)
fig,axs = plt.subplots(2,2)

axs[0,0].plot(x, y1,'go-', label='line 1', linewidth=2)
axs[0,0].set_title('Peak Disk Usage')
# axs[0,0].set_xlabel('Models')
axs[0,0].set_ylabel('MAE Loss (Test) in GB')
# axs[0,0].legend(['train', 'val'], loc='upper left')

axs[0,1].plot(x, y2, 'bo-', label='line 1', linewidth=2)
axs[0,1].set_title('Peak Simulation RAM')
# axs[0,1].set_xlabel('Models')
axs[0,1].set_ylabel('MAE Loss (Test) in GB')
# axs[0,0].legend(['train', 'val'], loc='upper left')

axs[1,0].plot(x, y3, 'ro-', label='line 1', linewidth=2)
axs[1,0].set_title('Peak Results Parsing RAM')
# axs[1,0].set_xlabel('Models')
axs[1,0].set_ylabel('MAE Loss (Test) in GB')
# axs[0,0].legend(['train', 'val'], loc='upper left')

axs[1,1].plot(x, y4, 'yo-', label='line 1', linewidth=2)
axs[1,1].set_title('Total Time Taken')
# axs[1,1].set_xlabel('Models')
axs[1,1].set_ylabel('MAE Loss (Test) in Secs')
# axs[0,0].legend(['train', 'val'], loc='upper left')